# Hypothesis Testing

Hypothesis testing (more formally "null hypothesis significance testing") is a central tool of statistical inference.

A formal hypothesis test involves two hypotheses, which must be stated up-front by the researcher:

**Null Hypothesis: $H_0$**

**Alternative Hypothesis: $H_a$**

The hypothesis tests that we have discussed in lecture are:

* Single population proportion test
* Difference between two population proportions test
* Single population mean test
* Difference between two population means test

In this tutorial, we will introduce how to carry out these hypothesis tests in Python using the Statsmodels package.

In [1]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import scipy.stats.distributions as dist

### Single population proportion test

_Example:_ In previous years, 52% of parents reported a belief that social media is the cause of their teenager’s lack of sleep. Do more parents today believe that their teenager’s lack of sleep is caused by social media? 

**Population**: Parents with a teenager (age 13-18)  
**Parameter of Interest**: p  
**Null Hypothesis:** p = 0.52  
**Alternative Hypthosis:** p > 0.52 (note that this is a one-sided test)

We interview 1018 parents (who constitute an IID sample from the population of interest), and 56% of these parents report that they believe that their teenager’s lack of sleep is caused by social media.

In this test, we take the null hypothesis proportion (0.52) to be a fixed, known value, not an estimate obtained from data (that therefore would be uncertain and inexact). The Statsmodels [proportions_ztest](https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportions_ztest.html) function returns a test statistic and corresponding p-value.  This test statistic is a Z-score, meaning that the further the test statistic falls from zero, the more evidence against the null hypothesis.  In this example, the p-value is much smaller than 0.05, and in most settings this would be considered to provide strong evidence against the null hypothesis.

In [2]:
n = 1018
pnull = .52
phat = .56
sm.stats.proportions_ztest(count=phat * n,       # number of success (1018 * 56/100)
                           nobs=n,               # number of trial or observations
                           value=pnull,          # null hypothesis
                           alternative='larger', # prop > value
                           prop_var=.52)         # variance of proportion

(2.5545334262132955, 0.005316510991822442)

### Difference between two population proportions test

_Example:_ Is there a difference between the population proportions of parents of Black children and parents of Hispanic children who report that their child has ever had swimming lessons?

**Populations**: All parents of Black children age 6-18 and all parents of Hispanic children age 6-18\
**Parameter of Interest**: p1 - p2, where p1 and p2 are the Black and Hispanic proportions, respectively \
**Null Hypothesis:** p1 - p2 = 0  \
**Alternative Hypthosis:** p1 - p2 $\neq$ 0

91 out of 247 (36.8%) sampled parents of Black children report that their child has had some swimming lessons.

120 out of 308 (38.9%) sampled parents of Hispanic children report that their child has had some swimming lessons.

The Statsmodels [test_proportions_2indep](https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.test_proportions_2indep.html) function returns a test statistic and p-value. In this example, the p-value is much greater than 0.05, so there is no evidence that the parents of Black and Hispanic children report differing levels of swimming lesson participation.

In [4]:
sm.stats.test_proportions_2indep(count1=91,   # Count for 1st sample
                                 nobs1=247,   # Sample size for 1st sample
                                 count2=120,  # Count for 2nd sample
                                 nobs2=308)   # Count for 

<class 'statsmodels.stats.base.HolderTuple'>
statistic = -0.5050754725272457
pvalue = 0.6135058278209921
compare = 'diff'
method = 'agresti-caffo'
diff = -0.021189336978810702
ratio = 0.9456140350877192
odds_ratio = 0.9138888888888889
variance = 0.001703246278498637
alternative = 'two-sided'
value = 0
tuple = (-0.5050754725272457, 0.6135058278209921)

### Single population mean test

_Research Question:_ Is the average cartwheel distance (in inches) for adults 
more than 80 inches?

**Population:** All adults \
**Parameter of Interest:** $\mu$, population mean cartwheel distance \
**Null Hypothesis:** $\mu$ = 80 \
**Alternative Hypthosis:** $\mu$ > 80 

We observe data from a sample of 25 adults, who have a sample mean of 82.46 and a sample standard deviation of 15.06.  Note that the hypothesized mean of 80 is considered to be an exact value not an estimate from a dataset which would have uncertainty associated with it.

In [5]:
df = pd.read_csv("Cartwheeldata.csv")
df.head()

,ID,Age,Gender,GenderGroup,Glasses,GlassesGroup,Height,Wingspan,CWDistance,Complete,CompleteGroup,Score
0,1,56,F,1,Y,1,62.0,61.0,79,Y,1,7
1,2,26,F,1,Y,1,62.0,60.0,70,Y,1,8
2,3,33,F,1,Y,1,66.0,64.0,85,Y,1,7
3,4,39,F,1,N,0,64.0,63.0,87,Y,1,10
4,5,27,M,2,N,0,73.0,75.0,72,N,0,4


In [6]:
n = len(df)
mean = df["CWDistance"].mean()
sd = df["CWDistance"].std()
(n, mean, sd)

(25, 82.48, 15.058552387264855)

Now we can carry out a formal test of the null hypothesis, using the Statsmodels [ztest](https://www.statsmodels.org/dev/generated/statsmodels.stats.weightstats.ztest.html#statsmodels.stats.weightstats.ztest) function, which returns a test statistic and p-value.  Since the p-value is 0.21, there is no evidence that the population mean differs from 80.

In [7]:
sm.stats.ztest(x1=df['CWDistance'],  # 1st of the two independent samples
               x2=None,              # 2nd of the two independent samples (default)
              value=80,              # one sample; x1 under null hupothesis, two samples; x1_mean - x2_mean - value
              alternative='larger')  # alternative hypothesis H_a > ??

(0.8234523266982029, 0.20512540845395266)

### Difference between two population means test

_Research Question:_ Considering adults in the NHANES data, do males have greater mean Body Mass Index than females?

**Population**: Adults in the NHANES data.  
**Parameter of Interest**: $\mu_1 - \mu_2$, Body Mass Index.  
**Null Hypothesis:** $\mu_1 = \mu_2$  
**Alternative Hypthosis:** $\mu_1 \neq \mu_2$

After we collect our data, we find the following sample means and sample standard deviations:

2976 Females 
$\mu_1 = 29.94$  
$\sigma_1 = 7.75$  

2759 Male Adults  
$\mu_2 = 28.78$  
$\sigma_2 = 6.25$  

$\mu_1 - \mu_2 = 1.16$